In [164]:
import pandas as pd
pd.set_option("mode.copy_on_write", True)

In [165]:
# Step 1: Get single family index from zillow per zip code

zillow_link = "https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv?t=1744845404"

# Step 2: Read the data and explore

home_price_data = pd.read_csv(zillow_link) # insert your path
home_price_data.info()
home_price_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26210 entries, 0 to 26209
Columns: 312 entries, RegionID to 2025-03-31
dtypes: float64(303), int64(3), object(6)
memory usage: 62.4+ MB


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,212578.371658,...,503061.748858,502677.970804,503091.888073,503792.345074,504933.153102,505665.837039,506094.852239,506526.985804,506203.975937,505287.023362
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,157063.441610,...,724974.515571,730397.609679,736030.643579,743231.954741,750308.179596,755455.822593,758298.843575,759390.057385,762508.399392,766757.158388
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,104837.094005,...,285278.490624,285044.015284,284862.823973,284674.593692,284255.947626,283463.010954,282729.921375,282116.635220,281487.998452,280648.469128
3,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,302826.826552,...,902354.276752,900992.689947,900271.455079,902846.916398,905005.396514,906207.426898,906285.019913,905673.895699,906185.800637,906574.960079
4,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,103964.199279,...,279422.672124,279124.980823,278854.581867,278607.090936,278102.350143,277502.474427,277164.994853,276925.883444,276524.721577,275660.628581


In [166]:
# Step 3: Rename region name by region zip code to avoid confusion
home_price_data.rename(columns={'RegionName': 'ZipCode'}, inplace=True)

# Step 4: Substet for California state
cali_home_price = home_price_data.loc[home_price_data["StateName"] == "CA"]

In [167]:
# Step 5: Subset post crisis and pre covid 2012 - 2019

# Keep only December 31 values from 2012 to 2019 to get yearly data
subset_date = cali_home_price.loc[:, cali_home_price.columns.str.match(r'^201[2-9]-12-31$')]

# Keep other columns
other_cols = ['RegionID','ZipCode', 'City', 'Metro', 'CountyName']
cali_home_price_2012_2019 = cali_home_price[other_cols + list(subset_date.columns)]

cali_home_price_2012_2019.head()


,RegionID,ZipCode,City,Metro,CountyName,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31
9,95992,90011,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,171374.048542,219389.288119,258472.315517,261365.155168,287415.816612,340439.079318,431419.090298,446867.836795
12,96193,90650,Norwalk,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,281441.392820,337896.092947,373010.031816,373556.602702,397209.484484,454366.024794,513432.471777,525969.212771
13,96361,91331,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,239370.723280,296319.593286,336781.152848,337580.772281,366298.039287,421416.484332,511899.970978,523787.160683
21,96025,90044,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,198149.615635,244612.884182,283844.090986,289512.722523,315115.956329,373281.393045,463544.130869,486036.904020
22,96817,92336,Fontana,"Riverside-San Bernardino-Ontario, CA",San Bernardino County,255540.617228,327905.368037,359340.324132,371407.715666,386342.160516,426207.560315,451557.267724,464034.147247


In [168]:
# Step 6: Melt the date columns so they become rows
home_price = cali_home_price_2012_2019.melt(
    id_vars=['RegionID', 'ZipCode', 'City', 'Metro', 'CountyName'],
    var_name='DATE',
    value_name='HOME_PRICE'
)


# Convert the date to datetime and extract the year
home_price['DATE'] = pd.to_datetime(home_price['DATE'])
home_price['YEAR'] = home_price['DATE'].dt.year

# Get only the latest value per year (December)
home_price = (
    home_price
    .sort_values(['ZipCode', 'YEAR', 'DATE'])
    .groupby(['ZipCode', 'YEAR'], as_index=False)
    .last()  # gets the most recent date for that year
)

# drop the date column to avoid confusion
home_price = home_price.drop(columns='DATE')


home_price.head(6)


,ZipCode,YEAR,RegionID,City,Metro,CountyName,HOME_PRICE
0,90001,2012,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,173120.150950
1,90001,2013,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,213862.137082
2,90001,2014,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,250721.453550
3,90001,2015,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,255205.837249
4,90001,2016,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,280380.216244
5,90001,2017,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,331520.073333


In [169]:
# Step 7: Adding change in price

#  Sort by ZIP and Year
home_price_sorted = home_price.sort_values(by=["ZipCode", "YEAR"])

# Group by ZIP and compute next year's price
home_price_sorted["NEXT_YEAR_PRICE"] = (
    home_price_sorted.groupby("ZipCode")["HOME_PRICE"].shift(-1)
)

# Compute the price change: next_year_price - current year price
home_price_sorted["PRICE_CHANGE"] = (
    home_price_sorted["NEXT_YEAR_PRICE"] - home_price_sorted["HOME_PRICE"]
)

# Compute PRECENT price change
home_price_sorted["PCT_PRICE_CHANGE (%)"] = (
    (home_price_sorted["NEXT_YEAR_PRICE"] - home_price_sorted["HOME_PRICE"])
    / home_price_sorted["HOME_PRICE"]
) * 100


# Drop rows where % change is undefined (e.g., HOME_PRICE = 0)
home_price_change_data = home_price_sorted.dropna(subset=["PCT_PRICE_CHANGE (%)"])


#  Drop rows where we can't compute a price change
home_price_change_data = home_price_sorted.dropna(subset=["PRICE_CHANGE"])

# Final check
home_price_change_data.shape
home_price_change_data.head(30)


,ZipCode,YEAR,RegionID,City,Metro,CountyName,HOME_PRICE,NEXT_YEAR_PRICE,PRICE_CHANGE,PCT_PRICE_CHANGE (%)
0,90001,2012,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,1.731202e+05,2.138621e+05,40741.986132,23.533936
1,90001,2013,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.138621e+05,2.507215e+05,36859.316468,17.235083
2,90001,2014,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.507215e+05,2.552058e+05,4484.383699,1.788592
3,90001,2015,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.552058e+05,2.803802e+05,25174.378994,9.864343
4,90001,2016,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.803802e+05,3.315201e+05,51139.857090,18.239467
5,90001,2017,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,3.315201e+05,4.156487e+05,84128.656186,25.376640
6,90001,2018,95982,Florence-Graham,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,4.156487e+05,4.354072e+05,19758.495110,4.753652
8,90002,2012,95983,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,1.619389e+05,2.037747e+05,41835.870518,25.834362
9,90002,2013,95983,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.037747e+05,2.386178e+05,34843.052638,17.098809
10,90002,2014,95983,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2.386178e+05,2.523322e+05,13714.391890,5.747431


In [170]:

# Save cleaned home price data

home_price_change_data.to_csv("final_home_price.csv", index=False)
